In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.alert import Alert

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# initiate driver and get website
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))




In [11]:
# Go to login https://www.benzinga.com/login
# Go to Analyst Ratings driver.get('https://www.benzinga.com/analyst-stock-ratings');
# Escape for popups logic
# Click performance
# Get URL and go back to analyst-stock-ratings
# Do for all analysts 

'III'

In [30]:
# Ranges from row 2 to row 1001

# Get the analyst's page
WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                    '//*[@id="__next"]/div[3]/div/div[1]/div[2]/div[1]/div/div[2]/div[2]/div/div[2]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[2]/td[7]/div/span[2]'))).click()



In [31]:
# Wait until page loads and get URL
# save url and row number as an id for now
driver.current_url
print("The current url is:"+str(get_url))

The current url is:https://www.benzinga.com/analyst-stock-ratings/analyst/580deb38846faa000106aea9


In [32]:
# Go back and do it again for each analyst

In [33]:
# After, create a process to iterate over the URLs and scrape the tables for ratings data per analysts
# analystRatingDate = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div[1]/div[2]/div[1]/div/div[2]/div[2]/div/div[2]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[2]/td[1]')

In [ ]:
# Save to pandas
# Save to postgres